# Data analisis de sentimientos BITCOIN

In [1]:
from newsapi import NewsApiClient
import pandas as pd

# 1. Iniciar el cliente con tu API Key
# IMPORTANTE: Reemplaza con tu clave real entre las comillas
newsapi = NewsApiClient(api_key='d7c0734c87d04ed69ea623c77e7406b3') 

def probar_conexion():
    print("📡 Probando conexión con NewsAPI...")
    try:
        # Hacemos una consulta pequeña y simple para probar
        response = newsapi.get_everything(
            q='Bitcoin',
            language='en',
            page_size=5  # Solo queremos 5 noticias para probar
        )
        
        # Verificamos si la respuesta es exitosa
        if response['status'] == 'ok':
            print(f"✅ Conexión EXITOSA. Total de resultados encontrados: {response['totalResults']}")
            
            # Mostramos el título de la primera noticia para confirmar
            primer_articulo = response['articles'][0]
            print(f"\n📰 Ejemplo de noticia recuperada:")
            print(f"- Título: {primer_articulo['title']}")
            print(f"- Fecha: {primer_articulo['publishedAt']}")
            return True
        else:
            print("❌ Error en la respuesta de la API.")
            return False

    except Exception as e:
        print(f"❌ Error al conectar: {e}")
        return False

# Ejecutar la prueba
probar_conexion()

📡 Probando conexión con NewsAPI...
✅ Conexión EXITOSA. Total de resultados encontrados: 11182

📰 Ejemplo de noticia recuperada:
- Título: Inside a Wild Bitcoin Heist: Five-Star Hotels, Cash-Stuffed Envelopes, and Vanishing Funds
- Fecha: 2025-11-17T10:00:00Z


True

In [2]:
# DEFINICIÓN DE PALABRAS CLAVE PARA LOS TRES EJES

# 1. Eje Bitcoin Directo (BTC)
# Se enfoca en el activo, el precio y el mercado cripto
KEYWORDS_BTC = "(bitcoin OR btc OR cryptocurrency OR 'crypto price' OR 'digital gold') NOT (fraud OR scam OR hack OR theft)"

# 2. Eje Tecnología Asociada (TECH)
# Se enfoca en la infraestructura y temas relacionados con la inversión en tecnología
KEYWORDS_TECH = "(blockchain OR web3 OR metaverse OR 'semiconductor shortage' OR 'AI investment' OR 'DeFi' OR 'mining rig')"

# 3. Eje Mercado Global/Macro (MACRO)
# Se enfoca en eventos que dictan la dirección económica global y el riesgo
KEYWORDS_MACRO = "(FED OR 'interest rate' OR inflation OR recession OR 'treasury bond' OR 'stock market crash' OR 'quantitative easing')"

# Rango de Fechas a Cubrir
# Es fundamental cubrir un rango amplio (ej. 1-2 años) para tener sufientes datos (60k+ filas)
FECHA_INICIO = '2025-11-12'  # Ejemplo: Último año
FECHA_FIN = '2025-12-12'    # Ejemplo: Hasta hace poco

print("✅ Palabras clave definidas para los 3 ejes.")

✅ Palabras clave definidas para los 3 ejes.


In [3]:
from datetime import datetime, timedelta
import time
from newsapi import NewsApiClient # Ya debe estar definido y conectado

# La fecha de fin se ajusta para que el bucle incluya el último día
# Convertimos las fechas string a objetos datetime
start_date = datetime.strptime(FECHA_INICIO, '%Y-%m-%d')
end_date = datetime.strptime(FECHA_FIN, '%Y-%m-%d') + timedelta(days=1) 

# Diccionario para almacenar los resultados de los 3 ejes
all_news_data = {
    'BTC': [],
    'TECH': [],
    'MACRO': []
}

# Mapeamos las keywords a los nombres de los ejes
keyword_map = {
    'BTC': KEYWORDS_BTC,
    'TECH': KEYWORDS_TECH,
    'MACRO': KEYWORDS_MACRO
}

current_date = start_date
print(f"--- INICIANDO EXTRACCIÓN DE NOTICIAS ---")
print(f"RANGO: {FECHA_INICIO} hasta {FECHA_FIN}")

# Bucle principal: Iterar día a día
while current_date < end_date:
    day_str = current_date.strftime('%Y-%m-%d')
    print(f"\nProcesando día: {day_str}")
    
    # 1. Iterar sobre los 3 ejes de búsqueda
    for axis_name, keyword_query in keyword_map.items():
        try:
            # Consulta a la API
            response = newsapi.get_everything(
                q=keyword_query,
                language='en',
                from_param=day_str, # Noticia publicada DESDE este día
                to=day_str,         # Noticia publicada HASTA este día
                sort_by='relevancy',
                page_size=100       # El máximo por página
            )
            
            # 2. Verificar el estado y guardar los datos
            if response['status'] == 'ok':
                count = len(response['articles'])
                print(f"  - {axis_name}: {count} artículos encontrados.")
                
                for article in response['articles']:
                    # Guardamos la data relevante (fecha, título, descripción) y el eje
                    all_news_data[axis_name].append({
                        'publishedAt': article['publishedAt'],
                        'title': article['title'],
                        'description': article['description'],
                        'source': article['source']['name'],
                        'axis': axis_name
                    })
            
            # Pausa breve para no saturar la API
            time.sleep(0.5) 

        except Exception as e:
            print(f"  ❌ ERROR en el eje {axis_name} el día {day_str}: {e}")
            # Pausa más larga si hay un error para evitar ser bloqueado
            time.sleep(5) 

    # Avanzamos al día siguiente
    current_date += timedelta(days=1)

print("\n--- EXTRACCIÓN FINALIZADA ---")
print(f"Total BTC: {len(all_news_data['BTC'])} artículos")
print(f"Total TECH: {len(all_news_data['TECH'])} artículos")
print(f"Total MACRO: {len(all_news_data['MACRO'])} artículos")

--- INICIANDO EXTRACCIÓN DE NOTICIAS ---
RANGO: 2025-11-12 hasta 2025-12-12

Procesando día: 2025-11-12
  - BTC: 99 artículos encontrados.
  - TECH: 45 artículos encontrados.
  - MACRO: 59 artículos encontrados.

Procesando día: 2025-11-13
  - BTC: 99 artículos encontrados.
  - TECH: 39 artículos encontrados.
  - MACRO: 69 artículos encontrados.

Procesando día: 2025-11-14
  - BTC: 97 artículos encontrados.
  - TECH: 41 artículos encontrados.
  - MACRO: 68 artículos encontrados.

Procesando día: 2025-11-15
  - BTC: 97 artículos encontrados.
  - TECH: 14 artículos encontrados.
  - MACRO: 19 artículos encontrados.

Procesando día: 2025-11-16
  - BTC: 97 artículos encontrados.
  - TECH: 11 artículos encontrados.
  - MACRO: 14 artículos encontrados.

Procesando día: 2025-11-17
  - BTC: 96 artículos encontrados.
  - TECH: 44 artículos encontrados.
  - MACRO: 53 artículos encontrados.

Procesando día: 2025-11-18
  - BTC: 98 artículos encontrados.
  - TECH: 32 artículos encontrados.
  - MACRO

In [4]:
# CONSOLIDACIÓN Y GUARDADO DE DATOS CRUDOS

# 1. Unir los diccionarios en una sola lista de artículos
full_articles_list = all_news_data['BTC'] + all_news_data['TECH'] + all_news_data['MACRO']

# 2. Convertir la lista a DataFrame de Pandas
df_raw_news = pd.DataFrame(full_articles_list)

# 3. Inspeccionar y guardar
print("ARTÍCULOS TOTALES RECUPERADOS:", df_raw_news.shape[0])
print("\nEJEMPLO DE DATOS CRUDOS:")
print(df_raw_news.head())

# Guardamos el dataset crudo antes de la limpieza
df_raw_news.to_csv('noticias_raw_sentimiento.csv', index=False)

print("\n✅ Datos crudos guardados en 'noticias_raw_sentimiento.csv'")

ARTÍCULOS TOTALES RECUPERADOS: 5576

EJEMPLO DE DATOS CRUDOS:
            publishedAt                                              title  \
0  2025-11-12T03:31:05Z  Bitcoin Bottomed At $98,000, Analyst Says: Tim...   
1  2025-11-12T21:31:08Z  Bitcoin User Accidentally Pays Over $105K To S...   
2  2025-11-12T19:00:00Z  Billionaire twins–backed stock surges 100% aft...   
3  2025-11-12T19:01:08Z  Take Profits On Bitcoin—It's 'Fall Season', Mo...   
4  2025-11-12T12:15:00Z            Demand Revival: Crypto Daybook Americas   

                                         description               source axis  
0  Bitcoin (CRYPTO: BTC) has likely established i...  Yahoo Entertainment  BTC  
1  A Bitcoin (CRYPTO: BTC) user paid a staggering...  Yahoo Entertainment  BTC  
2  Leap Therapeutics (Nasdaq: LPTX), a Cambridge,...            TheStreet  BTC  
3  Bitcoin (CRYPTO: BTC) tapped $105,000 before r...  Yahoo Entertainment  BTC  
4             The day ahead in crypto: Nov. 12, 2025            

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessearzate/bitcoin-and-us-treasury-with-daily-sentiment")

print("Path to dataset files:", path)

Path to dataset files: /home/vscode/.cache/kagglehub/datasets/jessearzate/bitcoin-and-us-treasury-with-daily-sentiment/versions/1


In [9]:
import os
import glob
import pandas as pd
# Asumimos que 'path' contiene la ruta de descarga del dataset

# 1. Mostrar la ruta para ver dónde se guardó
print(f"La ruta de descarga es: {path}")

# 2. Listar todos los archivos dentro de la carpeta descargada
# Nota: La carpeta descargada suele ser un ZIP que se descomprime, o una carpeta que contiene los archivos
archivos_en_path = os.listdir(path)
print(f"\nArchivos encontrados en la ruta: {archivos_en_path}")

# 3. Buscar y cargar el archivo CSV (el nombre variará)
csv_files = glob.glob(os.path.join(path, '*.csv'))

if not csv_files:
    # Si la descarga es un zip, a veces hay que mirar dentro de subcarpetas
    print("\n⚠️ No se encontraron archivos CSV directamente. Podría estar en una subcarpeta.")
    # Intenta buscar recursivamente (opcional)
    csv_files = glob.glob(os.path.join(path, '**', '*.csv'), recursive=True)
    
if csv_files:
    # Seleccionamos el primer CSV encontrado
    csv_file_path = csv_files[0]
    print(f"\n✅ Cargando archivo: {csv_file_path}")
    
    # Cargar el dataset en un DataFrame
    df_sentimiento = pd.read_csv(csv_file_path)
    
    # Imprimir información básica
    print("\n--- INFORMACIÓN DEL DATASET DE SENTIMIENTO ---")
    print("Dimensiones (filas, columnas):", df_sentimiento.shape)
    print("\nPrimeras 5 filas:")
    print(df_sentimiento.head())
    print("\nTipos de datos (dtypes):")
    print(df_sentimiento.info())
else:
    print("\n❌ Error: No se encontró ningún archivo CSV en la carpeta descargada.")

La ruta de descarga es: /home/vscode/.cache/kagglehub/datasets/jessearzate/bitcoin-and-us-treasury-with-daily-sentiment/versions/1

Archivos encontrados en la ruta: ['bitcoin_sentiment_12012022_11082025.csv']

✅ Cargando archivo: /home/vscode/.cache/kagglehub/datasets/jessearzate/bitcoin-and-us-treasury-with-daily-sentiment/versions/1/bitcoin_sentiment_12012022_11082025.csv

--- INFORMACIÓN DEL DATASET DE SENTIMIENTO ---
Dimensiones (filas, columnas): (1074, 28)

Primeras 5 filas:
   Unnamed: 0      timestamp      open      high       low     close  \
0           0  1669852800000  17165.44  17317.80  16855.00  16980.08   
1           1  1669939200000  16980.07  17108.25  16791.02  17094.71   
2           2  1670025600000  17094.25  17158.42  16863.58  16888.53   
3           3  1670112000000  16889.17  17199.99  16882.86  17108.90   
4           4  1670198400000  17108.90  17424.59  16865.22  16966.05   

         volume               datetime_utc  merge_date  domestic_series  ...  \
0

In [14]:

df_sentimiento.head()


,Unnamed: 0,timestamp,open,high,low,close,volume,datetime_utc,merge_date,domestic_series,...,total_non-marketable,treasury_bills,treasury_bonds,treasury_floating_rate_notes_(frn),treasury_inflation-protected_securities_(tips),treasury_notes,united_states_savings_inflation_securities,united_states_savings_securities,weighted_sentiment,sentiment_missing
0,0,1669852800000,17165.44,17317.80,16855.00,16980.08,31798.991518,2022-12-01 00:00:00+00:00,2022-12-01,7.577,...,2.135,3.456,3.012,4.104,0.487,1.68,10.148,2.694,0.408548,0
1,1,1669939200000,16980.07,17108.25,16791.02,17094.71,23096.436867,2022-12-02 00:00:00+00:00,2022-12-02,7.577,...,2.135,3.456,3.012,4.104,0.487,1.68,10.148,2.694,0.136171,0
2,2,1670025600000,17094.25,17158.42,16863.58,16888.53,14081.450672,2022-12-03 00:00:00+00:00,2022-12-03,7.577,...,2.135,3.456,3.012,4.104,0.487,1.68,10.148,2.694,-0.347766,0
3,3,1670112000000,16889.17,17199.99,16882.86,17108.90,16961.108288,2022-12-04 00:00:00+00:00,2022-12-04,7.577,...,2.135,3.456,3.012,4.104,0.487,1.68,10.148,2.694,0.502235,0
4,4,1670198400000,17108.90,17424.59,16865.22,16966.05,33618.451090,2022-12-05 00:00:00+00:00,2022-12-05,7.577,...,2.135,3.456,3.012,4.104,0.487,1.68,10.148,2.694,0.326272,0
